In [3]:
import pandas as pd

# --- Cấu hình ngưỡng (có thể chỉnh) ---
# Ví dụ:
# avg >= 8.0 -> "giỏi"
# 6.0 <= avg < 8.0 -> "khá"
# avg < 6.0 -> "yếu"
TH_GOOD = 8.0
TH_FAIR = 6.0

# --- Đọc file ---
df = pd.read_csv("../data_origin/udk_moodle_grades_course_670.csv",
                 names=["id","timemodified","userid","courseid","finalgrade","itemtype"],
                 header=None)  # Nếu file có header hãy đổi header=None phù hợp

# Nếu file đã có header (như bạn mô tả), dùng:
# df = pd.read_csv("udk_moodle_grades_course_670.csv")

# --- Chuẩn hoá điểm ---
# Thay -1 bằng 0 và thay NaN bằng 0
df["finalgrade"] = pd.to_numeric(df["finalgrade"], errors="coerce").fillna(0)
df.loc[df["finalgrade"] == -1, "finalgrade"] = 0

# --- Tính điểm trung bình cho mỗi user ---
user_avg = df.groupby("userid", as_index=False)["finalgrade"].mean()
user_avg = user_avg.rename(columns={"finalgrade": "avg_grade"})

# --- Gán cluster theo khoảng điểm ---
def assign_cluster(avg):
    if avg >= TH_GOOD:
        return "giỏi"
    elif avg >= TH_FAIR:
        return "khá"
    else:
        return "yếu"

user_avg["cluster"] = user_avg["avg_grade"].apply(assign_cluster)

# --- (Tùy chọn) Hiển thị các thông tin/tổng hợp ---
print("Số user theo cluster:")
print(user_avg["cluster"].value_counts())

print("\nMột vài dòng đầu:")
print(user_avg.head(10))

# --- Lưu kết quả ---
user_avg.to_csv("../data/output/user_avg_and_cluster.csv", index=False)
print("\n✅ Đã lưu kết quả vào file: user_avg_and_cluster.csv")

Số user theo cluster:
cluster
yếu     9
giỏi    9
khá     4
Name: count, dtype: int64

Một vài dòng đầu:
  userid  avg_grade cluster
0   7267   0.000000     yếu
1   8609   9.294118    giỏi
2   8670   9.000000    giỏi
3   8671   5.000000     yếu
4   8860   5.529412     yếu
5   8964   4.777778     yếu
6   8965   5.333333     yếu
7   9043   8.235294    giỏi
8   9109   9.117647    giỏi
9   9131   6.333333     khá

✅ Đã lưu kết quả vào file: user_avg_and_cluster.csv


In [5]:
import pandas as pd

# === Bước 1: Đọc dữ liệu ===
data = pd.read_csv("../data/simulate_grade_with_score.csv")

# === Bước 2: Tính điểm trung bình theo user_id ===
avg_scores = (
    data.groupby("user_id")
        .agg(
            avg_grade=("finalgrade", "mean"),
            cluster=("group", "first")  # Giữ nguyên cluster (group) đầu tiên của user
        )
        .reset_index()
)
# Nếu cần lưu ra file:
avg_scores.to_csv("user_avg_and_cluster_simulate.csv", index=False)